## Part of Speech (POS) Tagging
POS tagging is the process of marking up a word in a corpus to a corresponding part of a speech tag, based on its context and definition. This task is not straightfoward, as a particular word may have a different part of speech based on the context in which the word is used.

### The Different POS Tagging Techniques
1. **Lexical Based Methods** - Assign the POS tag the most frequently occurring with a word in the training corpus.
2. **Rule-Based Methods** - Assign POS tags based on rules. For example, we can have a rule that says, words ending with "ed" or "ing" must be assigned to a verb. Rule-Based Techniques can be used along with Lexical Based approaches to allow POS Tagging of words that are not present in the training corpus but are there in the testing data.
3. **Probabilistic Methods** - This method assigns the POS tags based on the probability of a particular tag sequence occurring. Conditional Random Fields (CRFs) and Hidden Markov Models (HMMs) are probabilistic approaches to assign a POS Tag.
4. **Deep Learning Methods** - Recurrent Neural Networks can also be used for POS tagging.